In [2]:
using LinearAlgebra
using SparseArrays

In [3]:
n = 6
M = 3
L = 2
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
Γ = .7
# RHS
B = randn(n, L);

In [4]:
# Initialize
X_σ     = zeros(n, L) 
X_exact = zeros(n, L)
Res     = zeros(M, L);

In [5]:
include("arnoldi.jl")
for l=1:L
    lhs = A - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
    Q, H = arnoldi(lhs, b, M)
    for m in 1:M
        s = vcat(norm(b), zeros(m, 1))
        z = H[1:m+1, 1:m] \ s
        x = Q[:, 1:m] * z
        Res[m, l] = norm(b - lhs*x)
        X_σ[:, l] = x
    end
end
norm(X_exact - X_σ)

0.0001357113382803524

In [6]:
include("bArnoldi.jl")
W, H = bArnoldi(A, B, M, L);

In [8]:
H_σ = H + Γ * vcat(Matrix(I, L*M, L*M), zeros(L, L*M));